In [3]:
!pip install spacy

     --------------------------------------- 12.2/12.2 MB 31.1 MB/s eta 0:00:00
     ------------------------------------- 480.9/480.9 kB 31.4 MB/s eta 0:00:00
     ---------------------------------------- 48.9/48.9 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 23.2 MB/s eta 0:00:00
     ------------------------------------- 181.6/181.6 kB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 94.7/94.7 kB ? eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 22.4 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 37.3 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import pprint
import csv
import datetime
import nltk
import requests
import csv
import json
import spacy
from spacy import displacy
import pprint

# nltk.download('punkt')

In [2]:
response = requests.get("https://apnews.com/article/george-santos-federal-charges-updates-33667a0900271e5002459ab748d8fdc8?utm_source=homepage&utm_medium=TopNews&utm_campaign=position_01")

APArticle = BeautifulSoup(response.content, 'html.parser')

#print(APArticle)

In [4]:
# Assemble basic dictionary about this news article

article_information = {}

In [6]:
# Find the canonical link
canonical_links = APArticle.find_all('link', rel='canonical')
print(canonical_links)

# Print the href attribute of each matching link
for link in canonical_links:
    self_URL = link.get('href')
    print(self_URL)
    
article_information["self_URL"] = self_URL

[<link data-rh="true" href="https://apnews.com/article/george-santos-federal-charges-updates-33667a0900271e5002459ab748d8fdc8" rel="canonical"/>]
https://apnews.com/article/george-santos-federal-charges-updates-33667a0900271e5002459ab748d8fdc8


In [7]:
# Find the Headline of the article

main_Headline = APArticle.find_all('h1')
main_Headline = main_Headline[0].text

article_information["headline"] = main_Headline

print(main_Headline)

George Santos pleads not guilty to federal indictment and says he won’t resign


In [8]:
# Find the datetime metadata for the article

published_time = APArticle.find('meta', {'property': 'article:published_time'}).get('content')
print(published_time)

modified_time = APArticle.find('meta', {'property': 'article:modified_time'}).get('content')
print(modified_time)

article_information["published_time"] = published_time
article_information["modified_time"] = modified_time

2023-05-10T13:20:32Z
2023-05-11T02:56:06Z


In [9]:
author_list = []

# Find who wrote the article
scripts = APArticle.find('script', attrs={'data-rh': 'true', 'type': 'application/ld+json'})
JSONscript = json.loads(scripts.text)
author_list = JSONscript['author']

article_information["author(s)"] = author_list

In [ ]:
#from nltk import Tree

#def to_nltk_tree(node):
#    if node.n_lefts + node.n_rights > 0:
#        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
#    else:
#        return node.orth_

#[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

In [10]:
article_information

{'self_URL': 'https://apnews.com/article/george-santos-federal-charges-updates-33667a0900271e5002459ab748d8fdc8',
 'headline': 'George Santos pleads not guilty to federal indictment and says he won’t resign',
 'published_time': '2023-05-10T13:20:32Z',
 'modified_time': '2023-05-11T02:56:06Z',
 'author(s)': ['Jake Offenhartz']}

In [11]:
image_data = []
image_url = JSONscript['image']
image_caption_div = APArticle.find_all('div', attrs={'data-key': 'embed-caption'})
image_caption = image_caption_div[0].text
image_attribution = "" #image_caption[-50:].replace(")", "").split("(")[1]

image_data.append([image_url, image_caption, image_attribution])
print(image_data)

[['https://storage.googleapis.com/afs-prod/media/5194e1e96809478c9dcf946d6bcab394/2000.jpeg', 'U.S. Rep. George Santos pleaded not guilty Wednesday to charges he duped donors, stole from his campaign and lied to Congress about being a millionaire, all while cheating to collect unemployment benefits. (May 10)', '']]


In [12]:
article_content = APArticle.find_all('div', {'class': 'Article', 'data-key': 'article'})

paragraphCount = 0
inner_para = article_content[0].find_all('p')

article_paragraphs = []

for index, para in enumerate(inner_para):
    
    paragraphCount += 1
    #print(index)
    
    inner_text = para.text 
    #print(inner_text)
    
    inner_links = para.find_all('a')
    content, href = "", ""
    
    # reset this array
    inner_link = []

    # Traverse the list to get the links and the associated text    
    for link in inner_links:
        content = link.string.strip()
        href = link['href']
        if href[0] == "/":
            href = "https://apnews.com" + href 
        inner_link.append([content, href])
        #print(content, href)
        
        para_ele = para.name
        
    article_paragraphs.append([index, inner_text, inner_link])

print(article_paragraphs)

article_information["mainContents"] = article_paragraphs

[[0, 'CENTRAL ISLIP, N.Y. (AP) — U.S. Rep. George Santos, infamous for fabricating his life story, pleaded not guilty Wednesday to charges he duped donors, stole from his campaign and lied to Congress about being a millionaire, all while cheating to collect unemployment benefits he didn’t deserve.', [['George Santos', 'https://apnews.com/hub/george-santos'], ['fabricating his life story', 'https://apnews.com/article/george-santos-lies-congress-new-york-republicans-9e4fd69b75894addb5bd2d4d5e8171ff']]], [1, 'Afterward, he said he wouldn’t drop his reelection bid and defied calls to resign.', []], [2, 'Santos’ 13-count federal indictment was a reckoning for a web of fraud and deceit that prosecutors say overlapped with the New York Republican’s fantastical public image as a wealthy businessman — a fictional biography that began to unravel after he won election last fall.', [['Santos’ 13-count federal indictment', 'https://www.documentcloud.org/documents/23809076-george-santos-indictment']

In [13]:
print(article_information["mainContents"][0][1])

CENTRAL ISLIP, N.Y. (AP) — U.S. Rep. George Santos, infamous for fabricating his life story, pleaded not guilty Wednesday to charges he duped donors, stole from his campaign and lied to Congress about being a millionaire, all while cheating to collect unemployment benefits he didn’t deserve.


In [14]:
nlp = spacy.load("en_core_web_sm")

abbreviated_paragraph = article_information["mainContents"][3][1]
#abbreviated_paragraph = article_information["mainContents"][5][1]

print(abbreviated_paragraph)
doc = nlp(abbreviated_paragraph)

Santos, 34, was released on $500,000 bond following his arraignment, about five hours after turning himself in to face charges of wire fraud, money laundering, theft of public funds and making false statements to Congress. He surrendered his passport and could face up to 20 years in prison if convicted.


In [24]:
!python -m spacy download en_core_web_sm

     --------------------------------------- 12.8/12.8 MB 34.4 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [15]:
displacy.render(doc, style="dep")

In [16]:
from spacy.matcher import Matcher

# Entity Finder
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}]
matcher.add("PROPER_NOUN", [pattern], greedy="LONGEST")
doc = nlp(abbreviated_paragraph)

In [17]:
matches = matcher(doc)
print(len(matches))

for match in matches[:100]:
    print(match, doc[match[1]:match[2]]) 

2
(451313080118390996, 0, 1) Santos
(451313080118390996, 40, 41) Congress


In [18]:
sentence_sense = []

for match in matches[:100]:
    #print(match, doc[match[1]:match[2]]) 
    
    for token in range(match[1], match[2]):
        try:
            currentWord = doc[token]
        
            #print(currentWord.i)
            #print(currentWord.text, currentWord.pos_, currentWord.dep_, currentWord.head.i, currentWord.head.pos_)

            # check for dependencies:
            if currentWord.dep_ == 'nsubj' and currentWord.head.pos_ == 'VERB' and currentWord.i in range(match[1], match[2]):
                print(f"'The {currentWord.pos_} '{currentWord.text}' has 'nsubj' dependency pointing to it. The word '{currentWord.head.text}' has '{currentWord.head.pos_}'")

                # Detail Parser
                parent = currentWord.head
                children = list(parent.children)
                print(children)
                for i, child in enumerate(children):

                    # action list

                    inner_list = []
                    print(i)
                    if child.dep_ == 'comp' or child.dep_ == 'conj':
                    # if child.dep_ == 'dobj' or child.dep_ == 'prep':

                        action_string = f"{child.text} - {child.pos_} - {child.dep_}"
                        inner_list.append(action_string)
                        print(inner_list)
                        print(child.children)
        except:
            continue
        
        token = match[1] + 1

In [19]:
from spacy.matcher import DependencyMatcher

# Entity and Verb Pair - Matcher Setup

matcherV = DependencyMatcher(nlp.vocab)

# Check for a Verb pointing to the Proper Noun
patternV1 = [
    {
        "RIGHT_ID": "verb_check", 
         "RIGHT_ATTRS": {"POS": "VERB"}
    },
    {
        "LEFT_ID": "verb_check",
        "REL_OP": ">",
        "RIGHT_ID": "proper_noun",
        "RIGHT_ATTRS": {"POS": "PROPN"},
        "RELATION": "nsubj"
    }    
]

# Reverse the order of words, and have the dependency be nsubjpass
patternV2 = [
    {
        "RIGHT_ID": "verb_check", 
         "RIGHT_ATTRS": {"POS": "VERB"}
    },
    {
        "LEFT_ID": "verb_check",
        "REL_OP": "<",
        "RIGHT_ID": "proper_noun",
        "RIGHT_ATTRS": {"POS": "PROPN"},
        "RELATION": "nsubjpass"
    }    
]

matcherV.add("PatternV1", [patternV1])
matcherV.add("PatternV2", [patternV2])
doc = nlp(abbreviated_paragraph)

In [20]:
action_output = []

# Entity and Verb Pair - Match Display and Storage
Verbmatches = matcherV(doc)
    
for match in Verbmatches:
    propn_token_index = match[1][1]
    propn_token  = doc[propn_token_index]
    verb_token = propn_token.head
    
    if match[1][0] > match[1][1]:
        action_output.append(f"{propn_token} - {propn_token.pos_} - {propn_token.dep_} --> {verb_token} - {verb_token.pos_} - {verb_token.dep_}")
    else:
        action_output.append(f"{verb_token} - {verb_token.pos_} - {verb_token.dep_} --> {propn_token} - {propn_token.pos_} - {propn_token.dep_}")

#print(action_output)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(action_output)

['Santos - PROPN - nsubjpass --> released - VERB - ROOT']


In [21]:

# Verb and Direct Object Pair - Matcher Setup

matcherW = DependencyMatcher(nlp.vocab)
patternW1 = [
    {
        "RIGHT_ID": "verb_check", 
         "RIGHT_ATTRS": {"POS": "VERB"}
    },
    {
        "LEFT_ID": "verb_check",
        "REL_OP": ">",
        "RIGHT_ID": "proper_noun",
        "RIGHT_ATTRS": {"POS": "*"},
        "RELATION": "dobj"
    }
]

patternW2 = [
    {
        "RIGHT_ID": "verb_check", 
         "RIGHT_ATTRS": {"POS": "VERB"}
    },
    {
        "LEFT_ID": "verb_check",
        "REL_OP": ">",
        "RIGHT_ID": "noun",
        "RIGHT_ATTRS": {"POS": "NOUN"},
        "RELATION": "dobj"
    }
]

matcherW.add("PatternW1", [patternW1])
matcherW.add("PatternW2", [patternW2])
doc = nlp(abbreviated_paragraph)

In [22]:
# Verb and Direct Object Pair - Display and Storage

verb_connect_output = []

VerbConnmatch = matcherW(doc)
    
for match in VerbConnmatch:
    propn_token_index = match[1][1]
    propn_token  = doc[propn_token_index]
    verb_token = propn_token.head
    #print(propn_token, propn_token.pos_, propn_token.dep_)
    #print(verb_token, verb_token.pos_, verb_token.dep_)
    
    if match[1][0] > match[1][1]:
        verb_connect_output.append(f"{propn_token} - {propn_token.pos_} - {propn_token.dep_} --> {verb_token} - {verb_token.pos_} - {verb_token.dep_}")
    else:
        verb_connect_output.append(f"{verb_token} - {verb_token.pos_} - {verb_token.dep_} --> {propn_token} - {propn_token.pos_} - {propn_token.dep_}")
        
#print(verb_connect_output)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(verb_connect_output)

[   'following - VERB - prep --> arraignment - NOUN - pobj',
    'face - VERB - advcl --> charges - NOUN - dobj',
    'making - VERB - conj --> statements - NOUN - dobj',
    'surrendered - VERB - ROOT --> passport - NOUN - dobj',
    'face - VERB - conj --> years - NOUN - dobj']


In [23]:
# Things (Nouns) and Verb Pair - Matcher Setup

matcherX = DependencyMatcher(nlp.vocab)
patternX = [
    {
        "RIGHT_ID": "noun_check", 
         "RIGHT_ATTRS": {"POS": "NOUN"}
    },
    {
        "LEFT_ID": "noun_check",
        "REL_OP": ">",
        "RIGHT_ID": "verb",
        "RIGHT_ATTRS": {"POS": "VERB"},
        "RELATION": "*"
    }
]

matcherX.add("PatternX", [patternX])
doc = nlp(abbreviated_paragraph)

In [24]:
# Verb and Direct Object Pair - Display and Storage

noun_verb_output = []

NounVerbmatch = matcherX(doc)
    
for match in NounVerbmatch:
    propn_token_index = match[1][1]
    propn_token  = doc[propn_token_index]
    verb_token = propn_token.head
    #print(propn_token, propn_token.pos_, propn_token.dep_)
    #print(verb_token, verb_token.pos_, verb_token.dep_)
    
    if match[1][0] > match[1][1]:
        noun_verb_output.append(f"{propn_token} - {propn_token.pos_} - {propn_token.dep_} --> {verb_token} - {verb_token.pos_} - {verb_token.dep_}")
    else:
        noun_verb_output.append(f"{verb_token} - {verb_token.pos_} - {verb_token.dep_} --> {propn_token} - {propn_token.pos_} - {propn_token.dep_}")
        
#print(noun_verb_output)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(noun_verb_output)

[]


In [26]:
sentence_one = article_information["mainContents"][8][1]
print(sentence_one)

doc = nlp(sentence_one)

# displacy.render(doc, style="dep")

Asked about Santos on Wednesday, Biden said, “I’m not commenting,” adding that anything he said would be construed by some interfering in the investigation. Asked if Congress should expel Santos, Biden said, “That’s for Congress to decide.”


In [27]:
# Thing and Possessive Pair - Matcher Setup

matcherZ = DependencyMatcher(nlp.vocab)
patternZ1 = [
    {
        "RIGHT_ID": "possessive_dep", 
         "RIGHT_ATTRS": {"DEP": "poss"}
    },
    {
        "LEFT_ID": "possessive_dep",
        "REL_OP": ">",
        "RIGHT_ID": "word",
        "RIGHT_ATTRS": {}
    }
]

patternZ2 = [
    {
        "RIGHT_ID": "y_check", 
         "RIGHT_ATTRS": {}
    },
    {
        "LEFT_ID": "y_check",
        "REL_OP": ">",
        "RIGHT_ID": "pronoun_check",
        "RIGHT_ATTRS": {"POS": "PRON"}
    }
]

matcherZ.add("PatternZ1", [patternZ1])
matcherZ.add("PatternZ2", [patternZ2])
doc = nlp(sentence_one)
print(doc)

Asked about Santos on Wednesday, Biden said, “I’m not commenting,” adding that anything he said would be construed by some interfering in the investigation. Asked if Congress should expel Santos, Biden said, “That’s for Congress to decide.”


In [28]:
# Thing and Possessive Pair - Matcher Setup

poss_output = []
possMatch = matcherZ(doc)
    
for match in possMatch:
    # print(match)
    propn_token_index = match[1][1]
    propn_token  = doc[propn_token_index]
    verb_token = propn_token.head
    
    # print(propn_token, propn_token.pos_, propn_token.dep_)
    # print(verb_token, verb_token.pos_, verb_token.dep_)
    
    if match[1][0] > match[1][1]:
        poss_output.append(f"{propn_token} - {propn_token.pos_} - {propn_token.dep_} --> {verb_token} - {verb_token.pos_} - {verb_token.dep_}")
    else:
        poss_output.append(f"{verb_token} - {verb_token.pos_} - {verb_token.dep_} --> {propn_token} - {propn_token.pos_} - {propn_token.dep_}")
        
#print(poss_output)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(poss_output)

[   'I - PRON - nsubj --> commenting - VERB - ccomp',
    'adding - VERB - advcl --> anything - PRON - dobj',
    'he - PRON - nsubj --> said - VERB - relcl',
    'That - PRON - nsubj --> ’s - VERB - ccomp']


In [29]:
# Quotation Finder  - Matcher Setup

quotation_object = []
article_quotations = []

matcherQ = Matcher(nlp.vocab)
patternQ = [{"ORTH": {"IN": ['"', '“']}},
           {"OP": "*"},
           {"ORTH": {"IN": ['"', '”']}}
          ]

matcherQ.add("PatternQ", [patternQ])

doc = nlp(sentence_one)
print(doc)

quote_output = []
quoteMatch = matcherQ(doc)

for match in quoteMatch:
    print(match, doc[match[1]:match[2]])
    article_quotations.append(doc[match[1]:match[2]])
    
if quoteMatch:
    entities = [[ent.text, ent.label_] for ent in doc.ents]
    print("Entities:", entities)

Asked about Santos on Wednesday, Biden said, “I’m not commenting,” adding that anything he said would be construed by some interfering in the investigation. Asked if Congress should expel Santos, Biden said, “That’s for Congress to decide.”
(3003386472110251488, 9, 16) “I’m not commenting,”
(3003386472110251488, 41, 50) “That’s for Congress to decide.”
(3003386472110251488, 9, 50) “I’m not commenting,” adding that anything he said would be construed by some interfering in the investigation. Asked if Congress should expel Santos, Biden said, “That’s for Congress to decide.”
Entities: [['Santos', 'PERSON'], ['Wednesday', 'DATE'], ['Biden', 'PERSON'], ['Congress', 'ORG'], ['Santos', 'PERSON'], ['Biden', 'PERSON'], ['Congress', 'ORG']]


In [30]:
# Extract all entities from the sentence
entities = [ent for ent in doc.ents]

# Find entities that are outside the quotation marks
filtered_entities = []
for entity in entities:
    entity_start = entity[0].idx
    entity_end = entity[-1].idx + len(entity[-1].text)
    inside_quotation = False

    for token in doc:
        # print(token.idx)
        if token.text in ['"', '“', '”']:
            if token.idx < entity_start and entity_start < token.idx + len(token.text):
                inside_quotation = True
                break
            if token.idx < entity_end - 1 and entity_end - 1 < token.idx + len(token.text):
                inside_quotation = True
                break

    if not inside_quotation:
        filtered_entities.append(entity)

# Print the filtered entities
# for entity in filtered_entities:
#     print("Entity:", entity.text)
#     print("Entity label:", entity.label_)
#     print("")
    
# print(filtered_entities)

quotation_object.append([article_quotations, filtered_entities])
print(quotation_object)

[[[“I’m not commenting,”, “That’s for Congress to decide.”, “I’m not commenting,” adding that anything he said would be construed by some interfering in the investigation. Asked if Congress should expel Santos, Biden said, “That’s for Congress to decide.”], [Santos, Wednesday, Biden, Congress, Santos, Biden, Congress]]]
